# EDA Summary & Key Findings

**Purpose:** Consolidate key insights from all EDA notebooks for project discussion and presentation.

**Notebooks Reviewed:**
1. Data Exploration
2. Data Quality
3. Univariate Analysis
4. Bivariate Analysis
5. Outlier Analysis
6. Multivariate Analysis


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

# Visualization settings
sns.set_style('whitegrid')
plt.rcParams['font.size'] = 12
pd.set_option('display.max_columns', None)


In [ ]:
# Load dataset
conn = sqlite3.connect('../databases/nhanes_1st.db')
df = pd.read_sql_query('SELECT * FROM raw_dataset', conn)
conn.close()

print(f"Dataset shape: {df.shape}")
print(f"Total records: {len(df):,}")
print(f"Total features: {df.shape[1]}")


## 1. Dataset Overview


In [ ]:
print("="*80)
print("DATASET OVERVIEW")
print("="*80)
print(f"\nTotal Records: {len(df):,}")
print(f"Total Features: {df.shape[1]}")
print(f"\nFeature Categories:")
print(f"  - Demographics: age, gender, ethnicity, income_ratio")
print(f"  - Physical Measurements: BMI, height, heart_rate")
print(f"  - Blood Tests: WBC, platelets, hemoglobin, MCV")
print(f"  - Kidney Markers: creatinine, uric_acid")
print(f"  - Liver Markers: AST, bilirubin, GGT, ALT")
print(f"  - Electrolytes: sodium, potassium")
print(f"  - Lipids: cholesterol")
print(f"  - Lifestyle: alcohol, smoking")
print(f"  - Target Variables: 4 tasks (CVD, Metabolic, Kidney, Liver)")


## 2. Data Quality Summary


In [ ]:
# Missing values summary
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({'Missing': missing, 'Percent': missing_pct})
missing_df = missing_df[missing_df['Missing'] > 0].sort_values('Percent', ascending=False)

print("="*80)
print("DATA QUALITY SUMMARY")
print("="*80)
print(f"\nColumns with missing values: {len(missing_df)}")
print(f"\nTop 10 columns with highest missing rates:")
print(missing_df.head(10).to_string())

high_missing = missing_df[missing_df['Percent'] > 50]
print(f"\n⚠️ Columns with >50% missing: {len(high_missing)}")
if len(high_missing) > 0:
    print(high_missing.to_string())


## 3. Target Variables Analysis (All 4 Tasks)


In [ ]:
# Task A: Cardiovascular Disease
print("="*80)
print("TASK A: CARDIOVASCULAR DISEASE")
print("="*80)
if 'has_cardiovascular_disease' in df.columns:
    cardio_counts = df['has_cardiovascular_disease'].value_counts(dropna=False)
    cardio_pct = df['has_cardiovascular_disease'].value_counts(normalize=True, dropna=False) * 100
    print(f"\nDistribution:")
    print(f"  Healthy (0): {cardio_counts.get(0, 0):,} ({cardio_pct.get(0, 0):.2f}%)")
    print(f"  Has CVD (1): {cardio_counts.get(1, 0):,} ({cardio_pct.get(1, 0):.2f}%)")
    print(f"  Missing: {df['has_cardiovascular_disease'].isnull().sum():,}")
    
    plt.figure(figsize=(8, 5))
    cardio_counts.plot(kind='bar', color=['skyblue', 'coral'])
    plt.title('Cardiovascular Disease Distribution')
    plt.xlabel('CVD Status')
    plt.ylabel('Count')
    plt.xticks([0, 1], ['Healthy', 'Has CVD'], rotation=0)
    plt.show()


In [ ]:
# Task B: Metabolic Syndrome Components
print("="*80)
print("TASK B: METABOLIC SYNDROME COMPONENTS (5 Labels)")
print("="*80)
metabolic_labels = [
    'high_waist_circumference',
    'high_triglycerides_mg_dl',
    'low_hdl_mg_dl',
    'high_blood_pressure',
    'high_glucose_mg_dl'
]

for label in metabolic_labels:
    if label in df.columns:
        counts = df[label].value_counts(dropna=False)
        pct = df[label].value_counts(normalize=True, dropna=False) * 100
        missing = df[label].isnull().sum()
        print(f"\n{label}:")
        print(f"  Normal (0): {counts.get(0, 0):,} ({pct.get(0, 0):.2f}%)")
        print(f"  Abnormal (1): {counts.get(1, 0):,} ({pct.get(1, 0):.2f}%)")
        print(f"  Missing: {missing:,} ({missing/len(df)*100:.2f}%)")


In [ ]:
# Task C: Kidney Function (ACR Log)
print("="*80)
print("TASK C: KIDNEY FUNCTION (ACR Log)")
print("="*80)
if 'kidney_acr_mg_g' in df.columns:
    kidney_data = df['kidney_acr_mg_g'].dropna()
    print(f"\nStatistics:")
    print(f"  Mean: {kidney_data.mean():.4f}")
    print(f"  Median: {kidney_data.median():.4f}")
    print(f"  Std: {kidney_data.std():.4f}")
    print(f"  Min: {kidney_data.min():.4f}")
    print(f"  Max: {kidney_data.max():.4f}")
    print(f"  Missing: {df['kidney_acr_mg_g'].isnull().sum():,} ({df['kidney_acr_mg_g'].isnull().sum()/len(df)*100:.2f}%)")
    
    plt.figure(figsize=(10, 5))
    plt.hist(kidney_data, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
    plt.axvline(kidney_data.mean(), color='red', linestyle='--', label=f'Mean: {kidney_data.mean():.2f}')
    plt.axvline(kidney_data.median(), color='green', linestyle='-', label=f'Median: {kidney_data.median():.2f}')
    plt.title('Kidney Function (ACR Log) Distribution')
    plt.xlabel('ACR (Log Scale)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()


In [ ]:
# Task D: Liver Function (ALT Log)
print("="*80)
print("TASK D: LIVER FUNCTION (ALT Log)")
print("="*80)
if 'liver_alt_U_L' in df.columns:
    liver_data = df['liver_alt_U_L'].dropna()
    print(f"\nStatistics:")
    print(f"  Mean: {liver_data.mean():.4f}")
    print(f"  Median: {liver_data.median():.4f}")
    print(f"  Std: {liver_data.std():.4f}")
    print(f"  Min: {liver_data.min():.4f}")
    print(f"  Max: {liver_data.max():.4f}")
    print(f"  Missing: {df['liver_alt_U_L'].isnull().sum():,} ({df['liver_alt_U_L'].isnull().sum()/len(df)*100:.2f}%)")
    
    plt.figure(figsize=(10, 5))
    plt.hist(liver_data, bins=50, color='coral', alpha=0.7, edgecolor='black')
    plt.axvline(liver_data.mean(), color='red', linestyle='--', label=f'Mean: {liver_data.mean():.2f}')
    plt.axvline(liver_data.median(), color='green', linestyle='-', label=f'Median: {liver_data.median():.2f}')
    plt.title('Liver Function (ALT Log) Distribution')
    plt.xlabel('ALT (Log Scale)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()


## 4. Key Findings & Insights


### 4.1 Data Quality Findings

1. **Missing Data:**
   - 4 columns have >50% missing values (smoking_status, high_glucose_mg_dl, high_triglycerides_mg_dl, alcohol_drinks_per_week)
   - Most clinical markers have 30-40% missing values
   - Missing data is handled via masking in the model

2. **Data Quality:**
   - Some impossible values detected (e.g., WBC >50, platelets outside normal range)
   - 20,489 duplicate rows found
   - Most outliers are medically valid (represent real medical conditions)

### 4.2 Target Variable Insights

1. **Cardiovascular Disease:**
   - Class imbalance present (needs to be addressed in training)
   - Missing values: ~40% of records

2. **Metabolic Syndrome:**
   - 5 binary labels (multi-label classification)
   - High missing rates for some components (glucose, triglycerides)
   - Each component can be analyzed independently

3. **Kidney & Liver Function:**
   - Regression tasks on log-transformed values
   - Right-skewed distributions (log transformation appropriate)
   - Missing values: ~20-40% of records

### 4.3 Feature Relationships

1. **High Correlations Found:**
   - Age ↔ Height (0.532)
   - Height ↔ BMI (0.494)
   - Liver AST ↔ Liver GGT (0.399)

2. **Clinical Insights:**
   - Age is a key factor for multiple conditions
   - Liver markers are correlated (expected clinically)
   - Physical measurements (height, BMI) are related

### 4.4 Outlier Analysis

1. **Outlier Detection:**
   - Z-score method: 6,859 outliers detected
   - IQR method: 26,228 outliers detected
   - Most outliers are medically valid

2. **Variables Needing Review:**
   - White blood cells count
   - Platelets count
   - Sodium
   - Potassium
   - Alcohol consumption

3. **Recommendations:**
   - Keep outliers for most variables (medically valid)
   - Use robust scaling methods
   - Consider log transformation for highly skewed variables


### 5.1 Data Preprocessing

1. **Handle Missing Values:**
   - Use masking approach (already implemented in dataset.py)
   - Consider imputation for features with <50% missing
   - Drop or flag features with >50% missing

2. **Feature Engineering:**
   - Log transformation already applied to ACR and ALT
   - Consider robust scaling (less sensitive to outliers)
   - Feature interactions (age × BMI, etc.)

3. **Outlier Handling:**
   - Keep medically valid outliers
   - Review and potentially remove outliers for: WBC, platelets, sodium, potassium
   - Use robust scaling methods

### 5.2 Model Training Considerations

1. **Class Imbalance:**
   - Use weighted loss functions for CVD prediction
   - Consider oversampling/undersampling techniques
   - Monitor precision-recall curves (not just accuracy)

2. **Multi-Task Learning:**
   - All 4 tasks share the same input features
   - Missing targets handled via masking
   - Consider task-specific loss weighting

3. **Evaluation Metrics:**
   - **CVD:** Accuracy, Precision, Recall, F1, ROC-AUC
   - **Metabolic:** Per-label metrics + micro-averaged metrics
   - **Kidney/Liver:** RMSE, MAE, R²

### 5.3 Validation Strategy

1. **Train/Test Split:**
   - Already implemented in Transformation.py
   - Ensure stratified split for CVD (maintain class balance)

2. **Cross-Validation:**
   - Consider k-fold CV for hyperparameter tuning
   - Monitor all 4 tasks during validation

3. **Early Stopping:**
   - Monitor validation loss for all tasks
   - Prevent overfitting on imbalanced classes


## 6. Summary for Project Discussion

### Key Points to Highlight:

1. **Dataset Size:** 56,893 records with 29 features
2. **Multi-Task Learning:** 4 different prediction tasks (1 binary, 1 multi-label, 2 regression)
3. **Data Quality:** Missing values handled via masking, outliers mostly medically valid
4. **Clinical Relevance:** All features are clinically meaningful health markers
5. **Challenges:** Class imbalance, missing data, multiple task types

### Strengths:

- Comprehensive health data from NHANES
- Well-structured multi-task learning setup
- Proper handling of missing values
- Medical context considered in outlier analysis

### Limitations:

- High missing rates for some features
- Class imbalance in CVD prediction
- Missing data patterns need investigation
- Some features have limited coverage

### Next Steps:

1. Train and evaluate the multi-task model
2. Analyze feature importance for each task
3. Compare single-task vs multi-task performance
4. Interpret model predictions in clinical context
